In [1]:
import tensorflow as tf
import numpy as np
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype(np.float32).reshape(-1, 28*28) / 255.0
x_test = x_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
x_valid, x_train = x_train[:5000], x_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [2]:
def shuffle_batch(x, y, batch_size):
    rnd_idx = np.random.permutation(len(x))
    n_batches = len(x) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        x_batch, y_batch = x[batch_idx], y[batch_idx]
        yield x_batch, y_batch

In [3]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
tf.set_random_seed(0)
np.random.seed(0)

In [4]:
lr = 0.01
epoch = 20
batch_size = 200
threshold = 1.0

In [5]:
n_inputs = 28 * 28  # MNIST
n_hidden1 = 300 # reused
n_hidden2 = 50  # reused
n_hidden3 = 50  # reused
n_hidden4 = 20  # new!
n_outputs = 10  # new!

x = tf.placeholder(tf.float32, shape=(None, n_inputs), name="x")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(x, n_hidden1, activation=tf.nn.relu, name="hidden1")       # reused
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2") # reused
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3") # reused
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4") # new!
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs")                         # new!

with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(entropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):
    opt = tf.train.GradientDescentOptimizer(lr)
    var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                   scope="hidden[34]|outputs")
    train = opt.minimize(loss, var_list=var_list)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [6]:
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                   scope="hidden[34]|outputs")

[<tf.Variable 'hidden3/kernel:0' shape=(50, 50) dtype=float32_ref>,
 <tf.Variable 'hidden3/bias:0' shape=(50,) dtype=float32_ref>,
 <tf.Variable 'hidden4/kernel:0' shape=(50, 20) dtype=float32_ref>,
 <tf.Variable 'hidden4/bias:0' shape=(20,) dtype=float32_ref>,
 <tf.Variable 'outputs/kernel:0' shape=(20, 10) dtype=float32_ref>,
 <tf.Variable 'outputs/bias:0' shape=(10,) dtype=float32_ref>]

In [7]:
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                   scope="hidden[34]")

[<tf.Variable 'hidden3/kernel:0' shape=(50, 50) dtype=float32_ref>,
 <tf.Variable 'hidden3/bias:0' shape=(50,) dtype=float32_ref>,
 <tf.Variable 'hidden4/kernel:0' shape=(50, 20) dtype=float32_ref>,
 <tf.Variable 'hidden4/bias:0' shape=(20,) dtype=float32_ref>]

In [8]:
tf.GraphKeys.GLOBAL_VARIABLES

'variables'

In [9]:
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # regular expression
reuse_vars

[<tf.Variable 'hidden1/kernel:0' shape=(784, 300) dtype=float32_ref>,
 <tf.Variable 'hidden1/bias:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'hidden2/kernel:0' shape=(300, 50) dtype=float32_ref>,
 <tf.Variable 'hidden2/bias:0' shape=(50,) dtype=float32_ref>,
 <tf.Variable 'hidden3/kernel:0' shape=(50, 50) dtype=float32_ref>,
 <tf.Variable 'hidden3/bias:0' shape=(50,) dtype=float32_ref>]

In [10]:
restore_saver = tf.train.Saver(reuse_vars) # to restore layers 1-3

In [11]:
init = tf.global_variables_initializer()

In [12]:
saver = tf.train.Saver()

In [13]:
with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./save/my_model_final.ckpt")

    for i in range(epoch):                                            # not shown in the book
        for x_batch, y_batch in shuffle_batch(x_train, y_train, batch_size): # not shown
            sess.run(train, feed_dict={x: x_batch, y: y_batch})        # not shown
        accuracy_val = accuracy.eval(feed_dict={x: x_valid, y: y_valid})     # not shown
        print(i, "Validation accuracy:", accuracy_val)                   # not shown

    save_path = saver.save(sess, "./save/my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./save/my_model_final.ckpt
0 Validation accuracy: 0.7826
1 Validation accuracy: 0.9198
2 Validation accuracy: 0.9372
3 Validation accuracy: 0.9448
4 Validation accuracy: 0.949
5 Validation accuracy: 0.9528
6 Validation accuracy: 0.9544
7 Validation accuracy: 0.955
8 Validation accuracy: 0.956
9 Validation accuracy: 0.9562
10 Validation accuracy: 0.9572
11 Validation accuracy: 0.9568
12 Validation accuracy: 0.956
13 Validation accuracy: 0.9572
14 Validation accuracy: 0.9566
15 Validation accuracy: 0.9556
16 Validation accuracy: 0.956
17 Validation accuracy: 0.9576
18 Validation accuracy: 0.9574
19 Validation accuracy: 0.9584
